In [9]:
import sys
import numpy as np
from matplotlib import pyplot
import torch
import torch.nn as nn
from torch import optim
from torch.utils.data import DataLoader
from torch.utils.data.dataset import Dataset
from icecream import ic
import pandas as pd

import random

ic("USING pytorch VERSION: ", torch.__version__)

ic| 'USING pytorch VERSION: ', torch.__version__: '1.6.0'


('USING pytorch VERSION: ', '1.6.0')

## Define a pytorch Dataset object to contain the training and testing data
Pytorch handles data shuffling and batch loading, as long as the user provides a "Dataset" class. This class is just a wrapper for your data that casts the data into pytorch tensor format and returns slices of the data. In this case, our data is in numpy format, which conveniently pytorch has a method for converting to their native format.

The init function takes the path to the csv and creates a dataset out of it. I actually have three different options here. The dataset could be composed such that x is the 'timestamp' of the movement,the previous set of angles, or a tuple of both.

In [18]:
class AngleDataset(Dataset):
    def __init__(self, x, y):
        x_dtype = torch.FloatTensor
        y_dtype = torch.FloatTensor     # for MSE or L1 Loss

        self.length = x.shape[0]

        self.x_data = torch.from_numpy(x).type(x_dtype)
        self.y_data = torch.from_numpy(y).type(y_dtype)

    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]

    def __len__(self):
        return self.length

def create_datasets(csv_path: str, train_perc: float = 0.8):
    df = pd.read_csv(csv_path)
    x_dtype = torch.FloatTensor
    y_dtype = torch.FloatTensor
    length = len(df)

    x_data = np.array([])
    y_data = np.array([])

    # if x = curr angles and y = next angles
    for i in range(length):
        if i < length - 1:
            x = np.array(df.iloc[i])
            y = np.array(df.iloc[i + 1])
        else:
            #since it loops anyway
            x = np.array(df.iloc[i])
            y = np.array(df.iloc[0])
        
        np.append(x_data, x)
        np.append(y_data,y)

    # if x = timestamp and y = angles
    # self.x_data = np.array(range(0, len(self.df)))
    # for i in range(self.length):
    #     self.y_data.append(self.df.iloc[i])
    
    # if x = both timestamp and curr_angles
    # timestamps = range(0, len(self.df))
    # for i in range(self.length):
    #     if i < self.length - 1:
    #         x = np.array(self.df.iloc[i])
    #         y = np.array(self.df.iloc[i + 1])
    #     else:
    #         #since it loops anyway
    #         x = np.array(self.df.iloc[i])
    #         y = np.array(self.df.iloc[0])
        
    #     self.x_data.append((timestamps[i], x))
    #     self.y_data.append(y)

    
    np.random.shuffle(x_data)
    np.random.shuffle(y_data)

    last_train_idx = int(len(x_data) * train_perc)

    train_x = x_data[:last_train_idx]
    train_y = y_data[:last_train_idx]
    test_x = x_data[last_train_idx:]
    test_y = y_data[last_train_idx:]

    return AngleDataset(x=train_x, y=train_y), AngleDataset(x=test_x, y=test_y)

train_dataset, test_dataset = create_datasets('./walk.csv')

        


## Define training methods for the model
These methods use an initialized model and training data to iteratively perform the forward and backward pass of optimization. Aside from some data reformatting that depends on the input, output, and loss function, these methods will always be the same for any shallow neural network.

In [6]:
def train_batch(model, x, y, optimizer, loss_fn):
    # Run forward calculation
    y_predict = model.forward(x)

    # Compute loss.
    loss = loss_fn(y_predict, y)

    # Before the backward pass, use the optimizer object to zero all of the
    # gradients for the variables it will update (which are the learnable weights
    # of the model)
    optimizer.zero_grad()

    # Backward pass: compute gradient of the loss with respect to model
    # parameters
    loss.backward()

    # Calling the step function on an Optimizer makes an update to its
    # parameters
    optimizer.step()

    return loss.data.item()


def train(model, loader, optimizer, loss_fn, epochs=5):
    losses = list()

    batch_index = 0
    for e in range(epochs):
        for x, y in loader:
            loss = train_batch(model=model, x=x, y=y, optimizer=optimizer, loss_fn=loss_fn)
            losses.append(loss)

            batch_index += 1

        if e % 50 == 0:
          ic("Epoch: ", e+1)
          ic("Batches: ", batch_index)

    return losses


## Define testing methods for the model
These methods are like training, but we don't need to update the parameters of the model anymore because when we call the test() method, the model has already been trained. Instead, this method just calculates the predicted y values and returns them, AKA the forward pass.


In [7]:
def test_batch(model, x, y):
    # run forward calculation
    y_predict = model.forward(x)

    return y, y_predict


def test(model, loader):
    y_vectors = list()
    y_predict_vectors = list()

    batch_index = 0
    for x, y in loader:
        y, y_predict = test_batch(model=model, x=x, y=y)

        y_vectors.append(y.data.numpy())
        y_predict_vectors.append(y_predict.data.numpy())

        batch_index += 1

    y_predict_vector = np.concatenate(y_predict_vectors)

    return y_predict_vector


## Define plotting method for loss
This is a plotting method for looking at the behavior of the loss over training iterations.

In [8]:
def plot_loss(losses, show=True):
    fig = pyplot.gcf()
    fig.set_size_inches(8,6)
    ax = pyplot.axes()
    ax.set_xlabel("Iteration")
    ax.set_ylabel("Loss")
    x_loss = list(range(len(losses)))
    pyplot.plot(x_loss, losses)

    if show:
        pyplot.show()

    pyplot.close()


## Define Model Architecture
- 12 inputs = 3 joint angles per leg, 4 legs
- 12 outputs = *same as above*


In [19]:
class PytorchBrain(torch.nn.Module):
    _id = 0 # Global genome identifier
    _num_inputs = 12
    _num_outputs = 12

    # @classmethod
    # def __get_new_id(cls):
    #     cls._id += 1
    #     return cls._id

    @classmethod
    def get_num_outputs(cls):
        return cls._num_outputs

    def __init__(self):
        super().__init__()
        
        # Inputs to hidden layer linear transformation
        self.h1 = torch.nn.Linear(PytorchBrain._num_inputs, 12)
        self.relu = torch.nn.ReLU() #output is same shape as input
        self.batch_norm = torch.nn.BatchNorm1d(12)
        self.h2 = torch.nn.Linear(12, 12)
        self.h3 = torch.nn.Linear(12, 12)

        # Output layer, 2 outputs to match to sine and cosine.
        self.output = torch.nn.Linear(12, PytorchBrain._num_outputs)
        
        # Define sigmoid activation
        self.sigmoid = torch.nn.Sigmoid()
        self.tanh = torch.nn.Tanh()
        
    def forward(self, x):
        # Pass the input tensor through each of our operations
        x = self.h1(x)
        x = self.relu(x)
        x = self.batch_norm(x)
        x = self.sigmoid(x)

        x = self.h2(x)
        x = self.relu(x)
        x = self.batch_norm(x)
        x = self.sigmoid(x)

        x = self.h3(x)
        x = self.relu(x)
        x = self.batch_norm(x)
        x = self.sigmoid(x)

        x = self.tanh(x)
        x = self.output(x)
        
        return x